In [1]:
import pandas as pd
import numpy as np
import sys
import os
import cv2

import matplotlib.pyplot as plt

from skimage import io,filters

In [71]:
GRAYED_DIR = "./outputs/Grayed_Image"
GRAYED_DIR2 = "./outputs/Croped_Image"
PLATE_SEG_CHARS = "./outputs/Plate_Seg_Chars"

对车牌二值化图像行和列扫描。当行扫描时跳变小于某个阈值时，则认为这行数据中不包含字符信息，这行的像素直接置0。这样去除掉上、下边框和铆钉的影响。当扫描完行之后，直接在前端和末端抹掉两列（经验值）信息。这是为了去掉左右边框的影响。

In [68]:
for i in range(1,151):
    path = os.path.join(GRAYED_DIR, str(i)+".png")
    img = io.imread(path, as_gray=True)
    
    rows, cols = img.shape
    idxs = np.concatenate((range(0,int(rows/10)), range(rows-int(rows/9),rows)))

    for k in idxs:
        sum_row = np.sum(img[k]) / 255 / cols
        if sum_row<0.35 or sum_row > 0.5:
            img[k] = 0

    _img = img[:, 5:-5]
    thresh = filters.threshold_otsu(_img)
    dst = (_img > thresh) * 1
    if not os.path.exists(GRAYED_DIR2):
        os.mkdir(GRAYED_DIR2)     
    path_out = os.path.join(GRAYED_DIR2, str(i)+".png")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)

字符截取

In [93]:
for i in range(1,151):
    path = os.path.join(GRAYED_DIR2, str(i)+".png")
 
    dir_out = PLATE_SEG_CHARS+"Plate_" +str(i)
    if not os.path.exists(dir_out):
        os.mkdir(dir_out)
 
    img = io.imread(path, as_gray=True)
    rows, cols = img.shape
    per = np.floor(cols/7.5).astype(int)
    for k in range(7):
        if k > 1:
            idxs = range(int((k+0.5)*per), int((k+1.5)*per))
        else:
            idxs = range(k*per, (k+1)*per)
        char = img[:,idxs]
        path_out = os.path.join(dir_out,str(k+1)+".png")
        io.imsave(path_out, char)
#         plt.figure()
#         io.imshow(char)
#         plt.show()